In [1]:
%pip install evaluate
%pip install rouge-score
%pip install transformers
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 22.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=02a96c992369cf42efb68809219be624551cefbcc6eebd67b5cb145854319264
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 39.4 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd drive/MyDrive/diff-lm/
%ls

/content/drive/MyDrive/diff-lm
base/  logs/                 modelsbo/    modelstabd-1/  wandb/
data/  models-base-uncased/  modelstabd/  samples.txt


In [2]:
import json
import os
# os.listdir('data')
# data = []
# with open("data/calendar.dev.jsonl") as f:
#     for line in f:
#         a=json.loads(line)
#         a["formula"] = a["formula"].replace("edu.stanford.nlp.sempre.overnight.SimpleWorld.", "")
#         data.append(a)

# print(data[0])

In [3]:
DOMAINS = (
    "calendar",
    "basketball",
    "blocks",
    "housing",
    "publications",
    "recipes",
    "restaurants",
    "socialnetwork",
)

def get_data(domain, dataset="train_with_dev"):
    data = []
    with open("data/" + domain + "." + dataset + ".jsonl") as f:
        for line in f:
            record = json.loads(line)
            record["formula"] = simplifier(record["formula"])
            data.append(record)
    return data

simplifier = lambda txt: txt.replace("edu.stanford.nlp.sempre.overnight.SimpleWorld.", "")
train_all = {}
test_all = {}
for domain in DOMAINS:
    train_all[domain] = get_data(domain)
    test_all[domain] = get_data(domain, dataset="test")

# train_all[DOMAINS[0]][0]
# dev_all[DOMAINS[0]][0]
test_all[DOMAINS[0]][0]

{'canonical': 'meeting whose end time is larger than 10am or 3pm',
 'formula': '(call listValue (call filter (call getProperty (call singleton en.meeting) (string !type)) (call ensureNumericProperty (string end_time)) (string >) (call ensureNumericEntity (call concat (time 10 0) (time 15 0)))))',
 'natural': 'which meetings end later than 10 in the morning or 3 in the afternoon'}

In [4]:
import numpy as np

def split_train_dev(domain, domains_data, train_size=200, remain_dev=0.2, shuffle=True):
  data = domains_data[domain]
  if shuffle:
    np.random.shuffle(data)
  size = len(data)
  dev_size = np.ceil((size - train_size) * 0.2).astype(int) + train_size
  return data[:train_size], data[train_size:dev_size]

train_dict = {}
dev_dict = {}
for domain in DOMAINS:
  train_dict[domain], dev_dict[domain] = split_train_dev(domain, train_all)

len(train_dict[DOMAINS[0]])

200

In [5]:
def prepare_data(data, shuffle=True):
  inputs = []
  outputs = []
  for domain in DOMAINS:
    domain_data = data[domain]
    if shuffle:
      np.random.shuffle(domain_data)
    for record in domain_data:
      inputs.append(record['natural'])
      outputs.append(record['canonical'])
  return inputs, outputs

In [6]:
import math
from torch.utils.data.dataset import Dataset
from transformers import AutoModelForPreTraining,AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import torch.nn.functional as F 
import random

In [8]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
%env WANDB_PROJECT=diff_lm_semantic_parsing

env: WANDB_PROJECT=diff_lm_semantic_parsing


In [9]:
training_args = TrainingArguments(
    report_to = 'wandb',  
    run_name="sample-diff",
    output_dir='./modelssample',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
    eval_accumulation_steps=10,
    label_names=['labels'],
)

In [10]:
class OvernightDataset(Dataset): 
    def __init__(self, data, init_model, max_len, func=prepare_data):
        self.tokenizer = AutoTokenizer.from_pretrained(init_model)
        self.inputs, self.labels = func(data)
        self.max_len = max_len
        self.tokenizer.model_max_length = max_len
    def __getitem__(self, index):
        from_tokenizer = self.tokenizer(self.inputs[index],padding="max_length",truncation = True,return_tensors="pt")
        label_tokens = self.tokenizer(self.labels[index],padding="max_length",truncation = True,return_tensors="pt")
        input_ids = from_tokenizer["input_ids"].squeeze_().long()
        ret_labels = label_tokens["input_ids"].squeeze_().long()
        token_type_ids = from_tokenizer["token_type_ids"].squeeze_().long()
        attention_mask = from_tokenizer["attention_mask"].squeeze_().long()
        labels_token_type_ids = label_tokens["token_type_ids"].squeeze_().long()
        labels_attention_mask = label_tokens["attention_mask"].squeeze_().long()
        # return input_ids,token_type_ids,attention_mask
        return {"input_ids": input_ids, 
                "token_type_ids" : token_type_ids, 
                "attention_mask" : attention_mask, 
                "labels" : ret_labels, 
                "labels_token_type_ids" : labels_token_type_ids, 
                "labels_attention_mask" : labels_attention_mask}
    def __len__(self):
        return len(self.labels)

In [11]:
class diffusion_bert(nn.Module):
    def __init__(self,init_model,max_len,max_step,k=1) -> None:
        super().__init__()
        if "bert-base" in init_model:
            self.model = AutoModelForMaskedLM.from_pretrained(init_model)
            freezed_w = [self.model.bert.embeddings.token_type_embeddings.weight,self.model.bert.embeddings.word_embeddings.weight] #self.model.bert.embeddings.LayerNorm.weight, self.model.bert.embeddings.LayerNorm.bias
        else:
            self.model = AutoModelForPreTraining.from_pretrained(init_model)
            freezed_w = [self.model.cls.seq_relationship.bias, self.model.cls.seq_relationship.weight, self.model.bert.pooler.dense.bias, self.model.bert.pooler.dense.weight, self.model.bert.embeddings.token_type_embeddings.weight,self.model.bert.embeddings.word_embeddings.weight] #self.model.bert.embeddings.LayerNorm.weight, self.model.bert.embeddings.LayerNorm.bias
        self.max_len = max_len
        self.max_step = max_step
        self.k=k
        self.time_embed = nn.Embedding(max_step,self.model.config.hidden_size)
        #self.layernorm = nn.LayerNorm(self.model.config.hidden_size, eps=self.model.config.layer_norm_eps)
        for p in  freezed_w:
            p.requires_grad = False
        # nn.init.constant_(self.time_embed.weight, 0)
    def forward(self,input_ids,token_type_ids,attention_mask, labels, labels_token_type_ids, labels_attention_mask, t=None):
        t = self.max_step
        input_shape = input_ids.size()
        seq_length = input_shape[1]
        
        position_ids = self.model.bert.embeddings.position_ids[:, 0 : seq_length]
        position_embeddings = self.model.bert.embeddings.position_embeddings(position_ids)


        # Trial 9:
        output_shape = labels.size()
        out_seq_length = output_shape[1]
        
        outpos_ids = self.model.bert.embeddings.position_ids[:, 0 : out_seq_length]
        out_pos_embeddings = self.model.bert.embeddings.position_embeddings(outpos_ids)

       
        with torch.no_grad():
            word_emb = self.model.bert.embeddings.word_embeddings(labels)
            inp_emb = self.model.bert.embeddings.word_embeddings(input_ids)
            #print(word_emb.shape)
            token_type_embeddings = self.model.bert.embeddings.token_type_embeddings(token_type_ids)
            labels_token_type_embeddings = self.model.bert.embeddings.token_type_embeddings(labels_token_type_ids)
        for t in range(1,self.max_step,self.k):
            with torch.no_grad():
                diffusion_steps = torch.ones(size = (output_shape[0],),device=input_ids.device).long()*t

                noise = torch.randn_like(word_emb)/math.sqrt(self.model.config.hidden_size)
                alpha = 1 - torch.sqrt((diffusion_steps+1)/self.max_step).view(-1,1,1)
                noisy_word = torch.sqrt(alpha)*word_emb+torch.sqrt(1-alpha)*noise + labels_token_type_embeddings
            
            time_embedding = self.time_embed(diffusion_steps).unsqueeze(1)
            noisy_word = inp_emb+noisy_word+position_embeddings+out_pos_embeddings+time_embedding
            
            #noisy_word = self.layernorm(noisy_word)
            noisy_word = self.model.bert.embeddings.LayerNorm(noisy_word)

            extended_attention_mask = self.model.bert.get_extended_attention_mask(labels_attention_mask, output_shape)
            
            encoder_outputs = self.model.bert.encoder(
                noisy_word,
                attention_mask=extended_attention_mask,
                head_mask=[None] * self.model.config.num_hidden_layers
            )
            word_emb = encoder_outputs[0]
        prediction_scores = self.model.cls.predictions(word_emb)
        loss = F.cross_entropy(prediction_scores.view(-1, self.model.config.vocab_size),labels.flatten(),ignore_index=0)
        
        #loss = F.smooth_l1_loss(sequence_output,word_emb)
        return loss,prediction_scores,labels

    @torch.no_grad()
    def sampler(self,device, batch, k=1):
        import time
        
        start_time = time.time()
        inp_ids = batch['input_ids']
        # inp_token_type_ids = batch['token_type_ids']
        inp_shape = inp_ids.size()
        N = inp_shape[0]
        inp_pos_ids = self.model.bert.embeddings.position_ids[:, 0 : inp_shape[1]]
        inp_position_embeddings = self.model.bert.embeddings.position_embeddings(inp_pos_ids)
        inp_emb = self.model.bert.embeddings.word_embeddings(inp_ids)
        # mean,std = stats
        # mean = torch.tensor(mean).view(1,3,1,1)
        # std = torch.tensor(std).view(1,3,1,1)    
        noisy_word = torch.normal(0,1,(N,self.max_len,self.model.config.hidden_size)).to(device) / math.sqrt(self.model.config.hidden_size)
        token_type_ids = torch.zeros(N,self.max_len).long().to(device)
        attention_mask = torch.ones(N,self.max_len).long().to(device)
        extended_attention_mask = self.model.bert.get_extended_attention_mask(attention_mask, attention_mask.shape)

        position_ids = self.model.bert.embeddings.position_ids[:, 0 : self.max_len]
        position_embeddings = self.model.bert.embeddings.position_embeddings(position_ids)
        token_type_embeddings = self.model.bert.embeddings.token_type_embeddings(token_type_ids)
        for t in range(self.max_step-1,0,-k):
        #for t in range(1999,0,-1):

            #prepare time emb
            diffusion_steps = torch.ones(size = (N,),device=device).long()*t
            time_embedding = self.time_embed(diffusion_steps).unsqueeze(1)

            model_input = inp_emb+noisy_word+inp_position_embeddings+position_embeddings+time_embedding
            model_input = self.model.bert.embeddings.LayerNorm(model_input)
            #denoise
            encoder_outputs = self.model.bert.encoder(
                model_input,
                attention_mask=extended_attention_mask,
                head_mask=[None] * self.model.config.num_hidden_layers
            )
            sequence_output = encoder_outputs[0]
            prediction_scores = self.model.cls.predictions(sequence_output)

            #clamp
            # pred = torch.argmax(prediction_scores,-1).long()
            # denoised_word = self.model.bert.embeddings.word_embeddings(pred)
            denoised_word = prediction_scores.softmax(-1) @ self.model.bert.embeddings.word_embeddings.weight.unsqueeze(0)
        
            #DDIM
            alpha_tk = 1 - math.sqrt((t+1-k)/self.max_step)#+1e-5
            alpha_t = 1 - math.sqrt((t+1)/self.max_step)+1e-5
            noise = (noisy_word - math.sqrt(alpha_t)*denoised_word)/math.sqrt(1-alpha_t)
            noisy_word = math.sqrt(alpha_tk)*(noisy_word/math.sqrt(alpha_t) + (math.sqrt((1-alpha_tk)/alpha_tk) - math.sqrt((1-alpha_t)/alpha_t))*noise)
            #noisy_word = math.sqrt(alpha_tk)*denoised_word + math.sqrt(1-alpha_tk)*noise
            print(f"\rnoise level {t}  {time.time()-start_time:.2f}",end='')
        
        # pred = torch.argmax(prediction_scores,-1).long()
        return prediction_scores

In [12]:
import evaluate
def compute_metrics(eval_preds):
    metric = evaluate.load('rouge')
    logits, labels = eval_preds
    # print(len(logits), len(logits[0]), len(logits[0][0]), len(logits[0][0][0]))
    # print(len(labels), len(labels[0]), len(labels[0][0]))
    # return
    predictions = np.argmax(logits[0], axis=-1)
    preds = [train_set.tokenizer.decode(s) for s in predictions]
    refs = [ train_set.tokenizer.decode(s) for s in labels]
    # lab = np.argmax(labels, axis=-1)
    return metric.compute(predictions=preds, references=refs)

In [13]:
max_len = 64
diff_step = 500
initializing = 'base/bert-tiny'#'base/bert-mini'
checkpoint='modelsforward/checkpoint-2000'
device = torch.device('cuda')
model = diffusion_bert(initializing,max_len,diff_step)
state = torch.load(checkpoint+'/pytorch_model.bin', map_location=device) #"/Saved_Models/20220903bert_diffusion/bestloss.pkl")

model_dict = model.state_dict()
# 1. filter out unnecessary keys
if list(state.keys())[0].startswith("module."):
    state = {k[7:]: v for k, v in state.items() if k[7:] in model_dict}
else:
    state = {k: v for k, v in state.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(state)
# 3. load the new state dict
model.load_state_dict(model_dict)

# model.load_state_dict(state,strict=True)
model = model.to(device)
model.eval()
print("Trial 1")

train_set = OvernightDataset(train_dict, init_model=initializing, max_len=max_len)
val_set = OvernightDataset(dev_dict, init_model=initializing, max_len=max_len)
test_set = OvernightDataset(test_all, init_model=initializing, max_len=max_len)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_set,         # training dataset
    eval_dataset=val_set,             # evaluation dataset
    compute_metrics=compute_metrics,
)

# trainer.train()
# wandb.finish()

# print("Start decoding")

# out = model.sampler(device, 10, 128)
# with open("samples.txt", 'w', encoding="utf-8") as f:
#     for s in out:
#         sample = test_set.tokenizer.decode(s.cpu().flatten())
#         f.write(sample+"\n")  


Trial 1


In [33]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_set, batch_size=64, shuffle=False)
batch = next(iter(test_dataloader))
for key, value in batch.items():
    batch[key] = batch[key].to(device)
batch['labels']

tensor([[ 101, 3116, 3005,  ...,    0,    0,    0],
        [ 101, 3116, 2008,  ...,    0,    0,    0],
        [ 101, 3116, 2008,  ...,    0,    0,    0],
        ...,
        [ 101, 3116, 2008,  ...,    0,    0,    0],
        [ 101, 3116, 2008,  ...,    0,    0,    0],
        [ 101, 3116, 3005,  ...,    0,    0,    0]])

In [14]:
trainer.predict(test_set)

***** Running Prediction *****
  Num examples = 2740
  Batch size = 64


PredictionOutput(predictions=(array([[[ -6.4487486,  -6.4717765,  -6.390377 , ...,  -8.297865 ,
          -7.6363983,  -9.740145 ],
        [-11.079183 ,  -8.414481 ,  -9.894846 , ..., -11.176077 ,
          -9.220392 , -15.00461  ],
        [-11.637106 ,  -8.330424 , -10.523021 , ..., -11.226908 ,
          -8.896508 , -13.099769 ],
        ...,
        [ -1.3059261,  -2.8618317,  -3.0105157, ...,  -3.2716618,
          -3.6331897, -10.676552 ],
        [ -1.6716211,  -2.817718 ,  -3.0843644, ...,  -3.193944 ,
          -3.6812348,  -9.965038 ],
        [ -2.993058 ,  -3.8146782,  -4.292628 , ...,  -4.578396 ,
          -4.9561768, -12.569873 ]],

       [[ -6.021119 ,  -5.29235  ,  -5.802695 , ...,  -7.0958443,
          -6.533963 ,  -8.92548  ],
        [ -9.264551 ,  -7.5554004,  -9.049974 , ...,  -9.642098 ,
          -7.500005 , -12.796541 ],
        [-11.420397 ,  -8.058542 ,  -9.849473 , ..., -10.440901 ,
          -8.336807 , -14.17404  ],
        ...,
        [ -2.2058437,  -

In [15]:
def prepare_domain_data(data, domain):
    inputs = []
    outputs = []
    domain_data = data[domain]
    for record in domain_data:
        inputs.append(record['natural'])
        outputs.append(record['canonical'])
    return inputs, outputs

for dom in DOMAINS:
    test_dom = OvernightDataset(test_all, init_model=initializing, max_len=max_len, func=lambda data : prepare_domain_data(data, dom))
    ret = trainer.predict(test_dom)
    print(dom, ":", ret.metrics)
    print()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file base/bert-tiny/config.json
Model config BertConfig {
  "_name_or_path": "base/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file base/bert-tiny/tokenizer.json. We won't load it.
Didn't find file base/bert-tiny/added_tokens.json. We won't load it.
Didn't find file base/bert-tiny/special_tokens_map.json. We won't load it.
Didn't find file base/bert-tiny/tokenizer_config.json. We won't load it

calendar : {'test_loss': 3.5671706199645996, 'test_rouge1': 0.09007748492903911, 'test_rouge2': 0.027197529575690743, 'test_rougeL': 0.08608808029758508, 'test_rougeLsum': 0.08621541635732544, 'test_runtime': 3.5922, 'test_samples_per_second': 46.769, 'test_steps_per_second': 0.835}



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file base/bert-tiny/config.json
Model config BertConfig {
  "_name_or_path": "base/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file base/bert-tiny/tokenizer.json. We won't load it.
Didn't find file base/bert-tiny/added_tokens.json. We won't load it.
Didn't find file base/bert-tiny/special_tokens_map.json. We won't load it.
Didn't find file base/bert-tiny/tokenizer_config.json. We won't load it

basketball : {'test_loss': 3.1895148754119873, 'test_rouge1': 0.17694963446913145, 'test_rouge2': 0.05365831278887924, 'test_rougeL': 0.1480707529634671, 'test_rougeLsum': 0.14831789341952656, 'test_runtime': 7.5283, 'test_samples_per_second': 51.937, 'test_steps_per_second': 0.93}



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file base/bert-tiny/config.json
Model config BertConfig {
  "_name_or_path": "base/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file base/bert-tiny/tokenizer.json. We won't load it.
Didn't find file base/bert-tiny/added_tokens.json. We won't load it.
Didn't find file base/bert-tiny/special_tokens_map.json. We won't load it.
Didn't find file base/bert-tiny/tokenizer_config.json. We won't load it

...

publications : {'test_loss': 3.7706563472747803, 'test_rouge1': 0.09800869461105599, 'test_rouge2': 0.031337958757749244, 'test_rougeL': 0.09131299470144041, 'test_rougeLsum': 0.09137695838929069, 'test_runtime': 3.4625, 'test_samples_per_second': 46.499, 'test_steps_per_second': 0.866}



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file base/bert-tiny/config.json
Model config BertConfig {
  "_name_or_path": "base/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file base/bert-tiny/tokenizer.json. We won't load it.
Didn't find file base/bert-tiny/added_tokens.json. We won't load it.
Didn't find file base/bert-tiny/special_tokens_map.json. We won't load it.
Didn't find file base/bert-tiny/tokenizer_config.json. We won't load it

recipes : {'test_loss': 3.2195627689361572, 'test_rouge1': 0.08183588054097404, 'test_rouge2': 0.025364197530864142, 'test_rougeL': 0.07721650353601632, 'test_rougeLsum': 0.07728143682560512, 'test_runtime': 4.5183, 'test_samples_per_second': 47.806, 'test_steps_per_second': 0.885}



Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file base/bert-tiny/config.json
Model config BertConfig {
  "_name_or_path": "base/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Didn't find file base/bert-tiny/tokenizer.json. We won't load it.
Didn't find file base/bert-tiny/added_tokens.json. We won't load it.
Didn't find file base/bert-tiny/special_tokens_map.json. We won't load it.
Didn't find file base/bert-tiny/tokenizer_config.json. We won't load it

restaurants : {'test_loss': 3.6618335247039795, 'test_rouge1': 0.09361389964741823, 'test_rouge2': 0.03019657674287078, 'test_rougeL': 0.08740519861917401, 'test_rougeLsum': 0.08750462505646275, 'test_runtime': 6.8926, 'test_samples_per_second': 48.167, 'test_steps_per_second': 0.87}

socialnetwork : {'test_loss': 3.819683074951172, 'test_rouge1': 0.09245456142784202, 'test_rouge2': 0.019669542266663516, 'test_rougeL': 0.0860951887253489, 'test_rougeLsum': 0.08603140416271508, 'test_runtime': 19.2141, 'test_samples_per_second': 46.008, 'test_steps_per_second': 0.729}



In [34]:
out = model.sampler(device, batch)
_, otpred, _ = model(**batch)
oot = torch.argmax(otpred,-1).long()
for i, s in enumerate(out[5:10]):
    sample = test_set.tokenizer.decode(s.cpu().flatten())
    org = test_set.tokenizer.decode(batch['labels'][i].cpu().flatten())
    ot = test_set.tokenizer.decode(oot[i].cpu().flatten())
    print()
    # print(sample)
    print("ot:", ot)
    print("org:", org)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
len(out)

128

In [ ]:
with torch.no_grad():
  emp = test_set.__getitem__(0)
  outputs = model(emp['input_ids'], emp['token_type_ids'], emp['attention_mask'], emp['labels'])
outputs

IndexError: ignored